In [160]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import json
import time
from datetime import datetime
from tqdm import tqdm

In [161]:
json_file = open("API_key.json")
API_KEY = json.load(json_file)["API_key"]
json_file.close()

REGION = 'euw1'
REGION_EU = 'europe'
watcher = LolWatcher(API_KEY)

### Seconds to hours, minutes and seconds

In [162]:
def sec_to_hms(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
      
    return "%02d minutes %02d seconds" % (minutes, seconds)

### Get ID of a user

In [163]:
def get_id(summoner_name, region='euw1'):
    player_id = watcher.summoner.by_name(region, summoner_name)["id"]
    return player_id

### Get puuid of a user

In [164]:
def get_puuid(summoner_name, region='euw1'):
    puuid = watcher.summoner.by_name(region, summoner_name)["puuid"]
    return puuid

### Get item name from ID

In [165]:
def get_item_name(item_id):
    item_version = watcher.data_dragon.versions_for_region(REGION)['n']['item']
    item_name = watcher.data_dragon.items(item_version)['data'][str(item_id)]['name']
    return item_name

### Add data for a specific champion from the history of a user

In [168]:
def add_data(Summoner_name):

    player_puuid = get_puuid(Summoner_name)
    my_matches = watcher.match.matchlist_by_puuid(REGION_EU,player_puuid)

    start_time = time.time()
    date = datetime.today().strftime('%d/%m/%Y')

    row = []
    champ_list = []
    item_list = []

    df = pd.read_csv('Lol_data.csv',encoding='cp1252')

    target_champ = 'Sett'
    matches_number = 20 #number of matches to analyze to found data, max = 20

    print("If the loading stops, it means that the API has reached the maximum number of requests.\n"+
          "You have to wait 2 minute before the loading resumes.\n"+
        "\nAdding "+target_champ+" data of the player "+Summoner_name+" :\n")

    for L in tqdm(range(matches_number)):

        for i in range(10): 
            try:
                test_match = watcher.match.by_id(REGION_EU, my_matches[L])
            except:
                pass
            champion = test_match['info']['participants'][i]['championName']
            champ_list = np.append(champ_list,champion) # get the list of champions in the game

        if target_champ in champ_list: # check if target champion is in the game
            target_champ_idx = np.where(champ_list == target_champ)[0][0]

            if target_champ_idx < 5: # get only the enemies 
                enemy_list = champ_list[5:10]
            else:
                enemy_list = champ_list[0:5]

            row = np.insert(enemy_list,0,target_champ)

            for k in range(6): # get the items of the target champion
                    try:
                        item_n_str = 'item'+str(k)
                        item = get_item_name(test_match['info']['participants'][target_champ_idx][item_n_str]) 
                        row = np.append(row,item) 

                    except:
                        row = np.append(row,'no item')

            row = np.append(row,[Summoner_name,player_puuid,date])

        if len(row):
            df_len = len(df)
            df.loc[df_len] = row

        champ_list = []
        row = []

    df.to_csv('Lol_data.csv',index=False,encoding='cp1252') # adding data to csv, encoding cp1252 for specific characters

    end_time = time.time()-start_time
    print("\n Data added in "+sec_to_hms(end_time))

In [169]:
add_data('EG PEIWAN Sao')

If the loading stops, it means that the API has reached the maximum number of requests.
You have to wait 2 minute before the loading resumes.

Adding Sett data of the player EG PEIWAN Sao :



100%|███████████████████████████████████| 20/20 [04:01<00:00, 12.05s/it]


 Data added in 04 minutes 01 seconds
